In [68]:
import pandas as pd
import numpy as np
import altair as alt
import os

In [69]:
projdir = r"C:\Users\michael.mccarthy\Resource Systems Group, Inc\Projects - 2024-Model & RTP Update-240130\3_Model\Data\Replica"
trips_path = os.path.join(projdir,'Auto trip list','replica-scog-06_18_25-trips_dataset.csv')

In [70]:
replica_trips = pd.read_csv(trips_path)

C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_30248\3314353167.py:1: DtypeWarning: Columns (24,25,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  replica_trips = pd.read_csv(trips_path)


In [71]:
replica_trips.columns

Index(['activity_id', 'origin_bgrp_2020', 'origin_trct_2020',
       'origin_cty_2020', 'origin_st_2020', 'destination_bgrp_2020',
       'destination_trct_2020', 'destination_cty_2020', 'destination_st_2020',
       'primary_mode', 'trip_purpose', 'previous_trip_purpose',
       'trip_start_time', 'trip_end_time', 'trip_duration_minutes',
       'trip_distance_miles', 'vehicle_type', 'transit_submode',
       'transit_agency', 'transit_route', 'origin_land_use',
       'origin_building_use', 'destination_land_use',
       'destination_building_use', 'trip_taker_person_id',
       'trip_taker_household_id', 'trip_taker_age', 'trip_taker_sex',
       'trip_taker_race_ethnicity', 'trip_taker_employment_status',
       'trip_taker_wfh', 'trip_taker_individual_income',
       'trip_taker_commute_mode', 'trip_taker_household_size',
       'trip_taker_household_income', 'trip_taker_available_vehicles',
       'trip_taker_resident_type', 'trip_taker_industry',
       'trip_taker_building_type

In [72]:
replica_trips.head()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,...,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020,origin_cty_fips_2020,destination_st_fips_2020,destination_cty_fips_2020,origin_st_fips_2020,origin_custom,destination_custom,tour_type
0,6586623149266995904,"1 (Tract 9702, Island, WA)","9702 (Island, WA)","Island County, WA",Washington,"0 (Tract 9901, Skagit, WA)","9901 (Skagit, WA)","Skagit County, WA",Washington,private_auto,...,"9901 (Skagit, WA)","Skagit County, WA",Washington,53029,53,53057,53,NaN,NaN,commute
1,9151891491960631727,"1 (Tract 9702, Island, WA)","9702 (Island, WA)","Island County, WA",Washington,"0 (Tract 9901, Skagit, WA)","9901 (Skagit, WA)","Skagit County, WA",Washington,private_auto,...,"9901 (Skagit, WA)","Skagit County, WA",Washington,53029,53,53057,53,NaN,NaN,commute
2,5952347108470654242,"1 (Tract 9518, Skagit, WA)","9518 (Skagit, WA)","Skagit County, WA",Washington,"0 (Tract 9901, Skagit, WA)","9901 (Skagit, WA)","Skagit County, WA",Washington,private_auto,...,"9901 (Skagit, WA)","Skagit County, WA",Washington,53057,53,53057,53,613.0,NaN,commute
3,1137916112793313995,"2 (Tract 9501, Skagit, WA)","9501 (Skagit, WA)","Skagit County, WA",Washington,"0 (Tract 9901, Skagit, WA)","9901 (Skagit, WA)","Skagit County, WA",Washington,private_auto,...,"9901 (Skagit, WA)","Skagit County, WA",Washington,53057,53,53057,53,429.0,NaN,commute
4,14285215277898054429,"2 (Tract 9701, Island, WA)","9701 (Island, WA)","Island County, WA",Washington,"0 (Tract 9901, Skagit, WA)","9901 (Skagit, WA)","Skagit County, WA",Washington,private_auto,...,"9901 (Skagit, WA)","Skagit County, WA",Washington,53029,53,53057,53,NaN,NaN,work_based


In [73]:
# code time of day
def codePeriods(data,todPeriods,todStart,todVar):
    data.loc[:, 'start_hour'] = data['trip_start_time'].str[:2].astype(int)
    # start HH:MM in hours, plus half of duration in hours 
    data.loc[:, 'mid_hour'] = np.floor(data['trip_start_time'].str[:2].astype(float) + (data['trip_start_time'].str[3:5].astype(float) / 60) + ((data['trip_duration_minutes']/2)/60))
    data.loc[:, todVar] = ''
    for i in range(len(todPeriods)):
        data.loc[:, todVar] = np.where(data['mid_hour'] >= todStart[i],todPeriods[i],data[todVar])

todPeriods = ["OP","AM","OP","PM","OP"] # Names of TOD periods
todStart = [0,7,9,14,17]
codePeriods(replica_trips,todPeriods,todStart,'time_period')


In [74]:
# process tracts outside of SCOG model
partial_tracts = ['9408.01 (Skagit, WA)','9511.01 (Skagit, WA)','9511.02 (Skagit, WA)','9512 (Skagit, WA)']
outside_tracts = ['9403.02 (Skagit, WA)','9501 (Skagit, WA)', '9901 (Skagit, WA)']
partial_or_out = partial_tracts # + outside_tracts

replica_trips.loc[~(replica_trips['origin_custom'].isna()),'model_origin'] = "Int"
replica_trips.loc[~(replica_trips['destination_custom'].isna()),'model_destination'] = "Int"

replica_trips.loc[(replica_trips['origin_cty_2020'] != 'Skagit County, WA'),'model_origin'] = "Ext"
replica_trips.loc[(replica_trips['destination_cty_2020'] != 'Skagit County, WA'),'model_destination'] = "Ext"

replica_trips.loc[(replica_trips['origin_custom'].isna()) & (replica_trips['origin_cty_2020'] == 'Skagit County, WA') &  (replica_trips['origin_trct_2020'].isin(partial_or_out)),'model_origin'] = "ExtSkagit"
replica_trips.loc[(replica_trips['origin_custom'].isna()) & (replica_trips['origin_cty_2020'] == 'Skagit County, WA') & ~(replica_trips['origin_trct_2020'].isin(partial_or_out)),'model_origin'] = "PossInt"

replica_trips.loc[(replica_trips['destination_custom'].isna()) & (replica_trips['destination_cty_2020'] == 'Skagit County, WA') &  (replica_trips['destination_trct_2020'].isin(partial_or_out)),'model_destination'] = "ExSkagit"
replica_trips.loc[(replica_trips['destination_custom'].isna()) & (replica_trips['destination_cty_2020'] == 'Skagit County, WA') & ~(replica_trips['destination_trct_2020'].isin(partial_or_out)),'model_destination'] = "PossInt"

In [75]:
# problems
replica_trips.loc[(replica_trips['origin_custom'].isna()) & (replica_trips['model_origin'] == "PossInt"), 'origin_trct_2020'].value_counts()

# seem like areas along water, so they should be in the model

origin_trct_2020
9407 (Skagit, WA)       166
9406 (Skagit, WA)        46
9501 (Skagit, WA)        42
9403.02 (Skagit, WA)     39
9901 (Skagit, WA)        21
9404.03 (Skagit, WA)     15
9527 (Skagit, WA)         8
9408.02 (Skagit, WA)      3
9508.02 (Skagit, WA)      1
Name: count, dtype: int64

In [81]:
# tag External markets
# External Visitor to Internal Work, Internal Resident to External Work, IE/EI Non-work, External-External, by Time of Day

replica_trips.loc[(replica_trips['model_origin'].isin(['Int','PossInt'])) & (replica_trips['model_destination'].isin(['Int','PossInt'])),'ext_type'] = "II"
replica_trips.loc[(replica_trips['model_origin'].isin(['Int','PossInt'])) &  ~(replica_trips['model_destination'].isin(['Int','PossInt'])),'ext_type'] = "IE"
replica_trips.loc[~(replica_trips['model_origin'].isin(['Int','PossInt'])) & (replica_trips['model_destination'].isin(['Int','PossInt'])),'ext_type'] = "EI"
replica_trips.loc[~(replica_trips['model_origin'].isin(['Int','PossInt'])) & ~(replica_trips['model_destination'].isin(['Int','PossInt'])),'ext_type'] = "EE"

# by purpose
replica_trips.loc[(replica_trips['trip_taker_home_cty_2020'] == 'Skagit County, WA') & (replica_trips['ext_type'] == "IE") & (replica_trips['trip_purpose'] == 'work'), 'ext_market'] = "IE Resident Work"
replica_trips.loc[~(replica_trips['trip_taker_home_cty_2020'] == 'Skagit County, WA') & (replica_trips['ext_type'] == "EI") & (replica_trips['trip_purpose'] == 'work'), 'ext_market'] = "EI Visitor Work"
replica_trips.loc[(replica_trips['ext_type'].isin(["IE","EI"])) & ~(replica_trips['trip_purpose'] == 'work'), 'ext_market'] = "IE/EI Non-work"
replica_trips.loc[(replica_trips['ext_type'] == "EE") , 'ext_market'] = "EE"
replica_trips.loc[(replica_trips['ext_type'] == "II") , 'ext_market'] = "II"

# by work location, direction doesn't matter
replica_trips.loc[(replica_trips['trip_taker_home_cty_2020'] == 'Skagit County, WA') & ~(replica_trips['trip_taker_work_cty_2020'] == 'Skagit County, WA') & (replica_trips['ext_type'].isin(["IE","EI"])) & (replica_trips['tour_type'] == 'commute'), 'ext_market2'] = "IE Resident Work"
replica_trips.loc[~(replica_trips['trip_taker_home_cty_2020'] == 'Skagit County, WA') & (replica_trips['trip_taker_work_cty_2020'] == 'Skagit County, WA') & (replica_trips['ext_type'].isin(["IE","EI"])) & (replica_trips['tour_type'] == 'commute'), 'ext_market2'] = "EI Visitor Work"
# replica_trips.loc[~(replica_trips['trip_taker_home_cty_2020'] == 'Skagit County, WA') & (replica_trips['ext_type'] == "EI") & (replica_trips['tour_type'] == 'commute'), 'ext_market2'] = "EI Visitor Work"
replica_trips.loc[(replica_trips['ext_type'].isin(["IE","EI"])) & ~(replica_trips['tour_type'] == 'commute'), 'ext_market2'] = "IE/EI Non-work"
# some commute trips for people who live and work in Skagit
replica_trips.loc[(replica_trips['ext_type'].isin(["IE","EI"])) & (replica_trips['tour_type'] == 'commute') & ~(replica_trips['ext_market2'].isin(["IE Resident Work","EI Visitor Work"])), 'ext_market2'] = "IE/EI Non-work"
replica_trips.loc[(replica_trips['ext_type'] == "EE") , 'ext_market2'] = "EE"



In [82]:
# hourly departures by market
replica_departures = replica_trips.groupby(['start_hour','ext_market']).size().reset_index(name='count')
replica_departures['hour_perc'] = replica_departures['count'] / replica_departures.groupby(['ext_market'])['count'].transform('sum')

ext_chart = alt.Chart(replica_departures).mark_line(point=False, color='#006FA1').encode(
    x='start_hour:O',
    y='hour_perc:Q',
    color='ext_market:N',
    tooltip=['start_hour', 'hour_perc']
)

ext_chart

alt.Chart(...)

In [ ]:
# hourly departures by market 2
# trying to find PM IE/EI work trips
replica_departures = replica_trips.groupby(['start_hour','ext_market2']).size().reset_index(name='count')
replica_departures['hour_perc'] = replica_departures['count'] / replica_departures.groupby(['ext_market2'])['count'].transform('sum')

ext_chart = alt.Chart(replica_departures).mark_line(point=False, color='#006FA1').encode(
    x='start_hour:O',
    y='hour_perc:Q',
    color='ext_market2:N',
    tooltip=['start_hour', 'hour_perc']
)

ext_chart

alt.Chart(...)

In [ ]:
# hourly departures by tour type
replica_departures = replica_trips.groupby(['start_hour','tour_type']).size().reset_index(name='count')
replica_departures['hour_perc'] = replica_departures['count'] / replica_departures.groupby(['tour_type'])['count'].transform('sum')

ext_chart = alt.Chart(replica_departures).mark_line(point=False, color='#006FA1').encode(
    x='start_hour:O',
    y='hour_perc:Q',
    color='tour_type:N',
    tooltip=['start_hour', 'hour_perc']
)

ext_chart

alt.Chart(...)

In [ ]:
# period PA factors for EI/IE
replica_factors = replica_trips.groupby(['time_period','ext_type','ext_market2']).size().reset_index(name='count')
#replica_factors.to_csv('replica_extmarket.csv',index=False)